In [ ]:
import sys
sys.path.append('..')

from src.vista3d.build_vista3d import build_vista3d_segresnet_decoder
from src.vista3d.modeling.segresnetds import SegResEncoder
import torch

/home/borntowarn/projects/chest-diseases/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = build_vista3d_segresnet_decoder()

spatial_dims 3
init_filters 48
in_channels 1
act ('relu', {'inplace': True})
norm ('instance', {'affine': True})
blocks_down (1, 2, 2, 4, 4)
anisotropic_scales None


In [ ]:
model.load_state_dict(torch.load("/home/borntowarn/projects/chest-diseases/training/weights/Vista3D/model.pt"))

<All keys matched successfully>

In [ ]:
class VistaEncoder(torch.nn.Module):
    def __init__(self, enc):
        super(VistaEncoder, self).__init__()
        self.encoder = enc
    def forward(self, x):
        return self.encoder(x)
enc = VistaEncoder(model.image_encoder.encoder.eval())
torch.save(enc.state_dict(), "/home/borntowarn/projects/chest-diseases/training/weights/vista3d/image_encoder.pt")

In [4]:
enc.cuda()

VistaEncoder(
  (encoder): SegResEncoder(
    (conv_init): Conv3d(1, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (layers): ModuleList(
      (0): ModuleDict(
        (blocks): Sequential(
          (0): SegResBlock(
            (norm1): InstanceNorm3d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
            (act1): ReLU(inplace=True)
            (conv1): Conv3d(48, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
            (norm2): InstanceNorm3d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
            (act2): ReLU(inplace=True)
            (conv2): Conv3d(48, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          )
        )
        (downsample): Conv3d(48, 96, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      )
      (1): ModuleDict(
        (blocks): Sequential(
          (0): SegResBlock(
            (norm1): Ins

In [6]:
z = torch.randn(2, 1, 128, 128, 128, device="cuda")

with torch.no_grad():
    x = enc(z)
    y = enc(z[:1])

for i, j in enumerate(x):
    print(torch.allclose(j[:1], y[i]))
    print(j[:1].shape, y[i].shape)


True
torch.Size([1, 48, 128, 128, 128]) torch.Size([1, 48, 128, 128, 128])
True
torch.Size([1, 96, 64, 64, 64]) torch.Size([1, 96, 64, 64, 64])
True
torch.Size([1, 192, 32, 32, 32]) torch.Size([1, 192, 32, 32, 32])
False
torch.Size([1, 384, 16, 16, 16]) torch.Size([1, 384, 16, 16, 16])
False
torch.Size([1, 768, 8, 8, 8]) torch.Size([1, 768, 8, 8, 8])


In [15]:
input_tensor = torch.arange(1*256*256*256, device="cuda").reshape(1, 1, 256, 256, 256).float()


In [16]:
# Инференс: разбиваем входной тензор на патчи 16x16x16, прогоняем через энкодер, собираем обратно

import torch
import torch.nn.functional as F
import time

with torch.no_grad():
    # Входной тензор
    patch_size = 64
    patches = []

    # Разбиваем на патчи
    # Эффективно разбиваем input_tensor на патчи с помощью unfold, чтобы избежать вложенных циклов и ускорить обработку
    # Размеры: (B, C, D, H, W)
    start_time = time.time()
    b, c, d, h, w = input_tensor.shape
    pad_d = (patch_size - d % patch_size) % patch_size
    pad_h = (patch_size - h % patch_size) % patch_size
    pad_w = (patch_size - w % patch_size) % patch_size

    # Дополняем тензор до кратного patch_size
    input_padded = F.pad(input_tensor, (0, pad_w, 0, pad_h, 0, pad_d))

    # Используем unfold для разбиения на патчи
    patches = input_padded.unfold(2, patch_size, patch_size)\
                         .unfold(3, patch_size, patch_size)\
                         .unfold(4, patch_size, patch_size)
    # patches shape: (B, C, num_patches_d, num_patches_h, num_patches_w, patch_size, patch_size, patch_size)
    # Переставляем оси и объединяем патчи в список
    print(patches.shape)
    patches = patches.permute(2, 3, 4, 0, 1, 5, 6, 7).reshape(-1, b, c, patch_size, patch_size, patch_size)
    print(patches.shape)
    # patches = [patch.squeeze(0) for patch in patches]  # убираем batch dim, если нужно

    end_time = time.time()
    print(f"Time taken to split patches: {end_time - start_time} seconds")

    start_time = time.time()
    processed_patches = []
    for i in patches:
        # print(i.shape)
        x = enc(i)

        for j in x:
            print(j.shape)
        
        processed_patches.append(torch.cat(
            [torch.mean(j, dim=(2, 3, 4)).cpu() for j in x],
            dim=1
        ))

    end_time = time.time()
    print(f"Time taken to process patches: {end_time - start_time} seconds")

torch.cuda.empty_cache()
res = torch.mean(torch.stack(processed_patches), dim=0)

torch.Size([1, 1, 4, 4, 4, 64, 64, 64])
torch.Size([64, 1, 1, 64, 64, 64])
Time taken to split patches: 0.002552032470703125 seconds
torch.Size([1, 48, 64, 64, 64])
torch.Size([1, 96, 32, 32, 32])
torch.Size([1, 192, 16, 16, 16])
torch.Size([1, 384, 8, 8, 8])
torch.Size([1, 768, 4, 4, 4])
torch.Size([1, 48, 64, 64, 64])
torch.Size([1, 96, 32, 32, 32])
torch.Size([1, 192, 16, 16, 16])
torch.Size([1, 384, 8, 8, 8])
torch.Size([1, 768, 4, 4, 4])
torch.Size([1, 48, 64, 64, 64])
torch.Size([1, 96, 32, 32, 32])
torch.Size([1, 192, 16, 16, 16])
torch.Size([1, 384, 8, 8, 8])
torch.Size([1, 768, 4, 4, 4])
torch.Size([1, 48, 64, 64, 64])
torch.Size([1, 96, 32, 32, 32])
torch.Size([1, 192, 16, 16, 16])
torch.Size([1, 384, 8, 8, 8])
torch.Size([1, 768, 4, 4, 4])
torch.Size([1, 48, 64, 64, 64])
torch.Size([1, 96, 32, 32, 32])
torch.Size([1, 192, 16, 16, 16])
torch.Size([1, 384, 8, 8, 8])
torch.Size([1, 768, 4, 4, 4])
torch.Size([1, 48, 64, 64, 64])
torch.Size([1, 96, 32, 32, 32])
torch.Size([1, 192

In [17]:
res

tensor([[ -150288.5938,  -524766.1250,    77602.2656,  ...,
          6388090.0000, -4845089.0000, 25343910.0000]])

In [31]:
input_tensor = torch.arange(1*256*256*256, device="cuda").reshape(1, 1, 256, 256, 256).float()


In [27]:
input_tensor = torch.arange(1*56*180*256, device="cuda").reshape(1, 1, 56, 180, 256).float()

import torch
from monai.transforms import (
    Compose,
    LoadImaged,
    ScaleIntensityRanged,
    Orientationd,
    Spacingd,
    CastToTyped,
    EnsureChannelFirstd
)

resample_to_spacing = (1.5, 1.5, 1.5)


monai_transforms = Compose([
    LoadImaged(
        keys=["image"],
        image_only=True,
    ),

    EnsureChannelFirstd(
        keys=["image"],
    ),

    ScaleIntensityRanged(
        keys="image",
        a_min=-963.8247715525971,
        a_max=1053.678477684517,
        b_min=0.0,
        b_max=1.0,
        clip=True
    ),

    Orientationd(
        keys=["image"],
        axcodes="RAS"
    ),

    Spacingd(
        keys=["image"],
        pixdim=resample_to_spacing,
        mode=["bilinear"],
        align_corners=True
    ),

    CastToTyped(
        keys=["image"],
        dtype=[torch.float32]
    )
])

file = '/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/train_fixed/train_7/train_7_a/train_7_a_1.nii.gz'
input_tensor = monai_transforms({"image": file})['image'].as_tensor().unsqueeze(0).cuda()

In [14]:
input_tensor.shape

torch.Size([1, 1, 232, 232, 190])

In [40]:
x_encoded = enc(torch.randn(1, 1, 1, 1, 32, device="cuda"))

In [ ]:
b, c, d, h, w = input_tensor.shape
patch_size = 64
processed_patches = []
with torch.no_grad():
    for z in range(0, d, patch_size):
        for y in range(0, h, patch_size):
            for x in range(0, w, patch_size):
                patch = input_tensor[
                    :, :, 
                    z:min(z + patch_size, d),
                    y:min(y + patch_size, h),
                    x:min(x + patch_size, w)
                ]

                dz, dy, dx = patch.shape[2:]

                # если все измерения < 32 → паддим первую ось (d) до 32
                if dz < 32 and dy < 32 and dx < 32:
                    pad_d = 32 - dz
                    patch = torch.nn.functional.pad(patch, (0, 0, 0, 0, 0, pad_d))  # паддим только по глубине
                
                print(patch.shape)

                x_encoded = enc(patch)
                processed_patches.append(
                    torch.cat([torch.mean(j, dim=(2, 3, 4)).cpu() for j in x_encoded], dim=1)
                )

res = torch.mean(torch.cat(processed_patches, dim=0), dim=0, keepdim=True)
res

torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 1])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 1])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 1])
torch.Size([1, 1, 64, 3, 64])
torch.Size([1, 1, 64, 3, 64])
torch.Size([1, 1, 64, 3, 64])
torch.Size([1, 1, 64, 3, 1])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 1])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 1])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 64])
torch.Size([1, 1, 64, 64, 1])
torch.Size([1, 1, 64, 3, 64])
torch.Size([1, 1, 64, 3, 64])
torch.Size([1, 1, 64, 3, 64])
torch.Size([1, 1, 64, 3, 1])
torch.Size([1, 1, 64, 64

tensor([[ 0.1843,  0.3107,  0.0768,  ...,  0.3355,  0.3752, -0.1387]])

: 

In [42]:
from monai.utils import (
    fall_back_tuple,
)

batch_size, _, *image_size_ = input_tensor.shape
roi_size = (128, 128, 128)
roi_size = fall_back_tuple(roi_size, image_size_)
pad_size = []
for k in range(len(input_tensor.shape) - 1, 1, -1):
    diff = max(roi_size[k - 2] - input_tensor.shape[k], 0)
    half = diff // 2
    pad_size.extend([half, diff - half])

In [33]:
res.shape

torch.Size([1, 1488])

In [21]:
processed_patches[0].shape

torch.Size([1, 1488])